# Configure 3-Stage Clos Network

Modern cloud-scale data center networks require increased server-to-server communication over a network that stays resilient despite the rapid increase in the number of devices.

The 3-stage Clos network is a robust IP-BGP underlay network that enables servers in a large-scale data center to communicate with each other with minimum latency. 

The following topology diagram depicts a 3-stage Clos network with two leaf routers in Tier-0 and two spine routers in Tier-1. This notebook shows you how to configure 3-stage Clos network on this topology.

<br/><br/>

<center><img src="./images/522559-2.jpg" width="700"/></center>

<br/><br/>

A 3-stage Clos network consists of data center network switches where each spine switch connects to all leaf switches. And each leaf switch connects to a rack of servers in the data-center. Any server in the data-center is just three hops away from another server. The first hop is from the server to the directly connected leaf switch, the second hop is across the spine switches to the destination leaf switch, and the third hop is between the destination leaf switch to the destination server. This network architecture is highly scalable. Also, irrespective of the number of devices in the data center, the hops between the servers (or the end-hosts) is always 3, ensuring consistent latency in the data-center network.

Switches and routers can be used interchangeably in a 3-stage Clos network. This notebook demonstrates how to configure 3-Stage Clos Network using Cisco 8000 series routers that run SONiC. 

In this notebook:
- The Cisco 8000 router variant 8102-64H are the Tier 1 routers - Spine0 (S0) and Spine1 (S1).
- The Cisco 8000 router variant 8101-32H are the Tier 0 routers - Leaf0 (L0) and Leaf1 (L1).



As you read through this notebook, click the buttons in each section to send configuration commands to the devices in the simulated network as shown in the above topology diagram. This notebook walks you through the following sections to configure a 3 stage Clos network:
* [Prerequistes](#prerequisites)
* [Access Devices](#access-devices) 
* [Configure Host Names](#Configure-Host-Names)
* [Assign IP-Addresses](#Assign-IP-Addresses)
* [Configure eBGP](#Configure-eBGP)
* [Send Traffic from TREX](#Send-Traffic-from-TREX)
* [Verify Traffic Statistics](#Verify-Traffic-Statistics)
* [Clean up Router Configurations](#Clean-up-Router-Configurations)


### Prerequisites

Before you begin with this notebook, reserve the Cisco 8000 SONiC Notebook sandbox and connect to the Sandbox VPN. Proceed to step 3 if you already have an active reservation for [Cisco 8000 SONiC Notebook](https://devnetsandbox.cisco.com/RM/Diagram/Index/219b721f-4116-4e47-adfa-c41ab540ca22?diagramType=Topology). 

1. To reserve the Cisco 8000 SONiC Notebook Sandbox, access [this link](https://devnetsandbox.cisco.com/RM/Diagram/Index/219b721f-4116-4e47-adfa-c41ab540ca22?diagramType=Topology) and click the **RESERVE** button on the top-right corner of the page. The setup takes 25–30 minutes.
2. You receive the software VPN credentials via email when the Sandbox is ready.
3. Enter the VPN credentials in the notebooks in this Learning Lab to connect to the Cisco 8000 Emulator on the Sandbox.

For example:  

```
Enter the Lab Network Address provided in the Sandbox Lab email: devnetsandbox-reservation.cisco.com:nnnnn
Enter the username provided in the Sandbox Lab email: xxxx
Enter the password provided in the Sandbox Lab email: yyyyy

(Click the Enter key on your keyboard after every entry)

```


In [ ]:
import os
os.environ['VPN_SERVER'] = input('Enter the Lab Network Address provided in the Sandbox Lab email: ')
os.environ['VPN_USERNAME'] = input('Enter the username provided in the Sandbox Lab email: ')
os.environ['VPN_PASSWORD'] = input('Enter the password provided in the Sandbox Lab email: ')

Click the following buttons to start a VPN connection to the Cisco 8000 SONiC Notebook Sandbox. You will not see any output after you click this button, because VPN access runs in the background. After clicking this button once, you can continue by clicking the next button.

In [5]:
%%bash --bg
startvpn.sh

View the log details of the VPN connection.

In [ ]:
!cat /var/log/openconnect/openconnect.log

This Learning Lab is now connected to the Sandbox VPN. Click the buttons in the following sections to configure a 3-stage Clos network. 

> **Note**: To view or edit the code that executes when you click the buttons, click the hide-code toggle-button in the top panel of this notebook.

### Access Devices
Click the following button to access the SSH console of each of the 4 routers and traffic generator in the topology. 

In [ ]:
from lib.leaf_spine import *
nodes = {
         'S0':'', 
         'S1':'',
         'L0':'', 
         'L1':'', 
         'trex':''
        }
tb = access_device_consoles("lib/leaf_spine.yaml", nodes)

### Configure Host Names

This section configures host names for the spine and leaf routers for easy identification.

The following commands configures the host name and saves the configurations:
```
sudo config hostname <host-name>
sudo config save -y
```

In [ ]:
out = nodes['S0'].execute('sudo config hostname SPINE0')
out = nodes['S0'].execute('sudo config save -y')
out = nodes['S1'].execute('sudo config hostname SPINE1')
out = nodes['S1'].execute('sudo config save -y')
out = nodes['L0'].execute('sudo config hostname LEAF0')
out = nodes['L0'].execute('sudo config save -y')
out = nodes['L1'].execute('sudo config hostname LEAF1')
out = nodes['L1'].execute('sudo config save -y')

### Assign IP Addresses

This step assigns IPv4 addresses on the interfaces of the routers and traffic generator as per the topology diagram. 

The following commands configures IPv4 addresses on the router interfaces and displays the configured values:

    sudo config interface ip add <interface> <IPv4 address> 
    show ip interfaces

In [ ]:
print ("\n--- Configuring S0 ---")
out = nodes['S0'].execute('sudo config interface ip add Ethernet0 10.0.1.1/24')
out = nodes['S0'].execute('sudo config interface ip add Ethernet8 10.0.2.1/24')
out = nodes['S0'].execute('sudo config interface ip add Loopback0 10.10.10.100/32')
out = nodes['S0'].execute('sudo config save -y')
print ("\n--- Verify IP address on S0 ---")
out = nodes['S0'].execute('show ip interfaces')  

In [ ]:
print ("\n--- Configuring S1 ---")
out = nodes['S1'].execute('sudo config interface ip add Ethernet0 10.0.3.1/24')
out = nodes['S1'].execute('sudo config interface ip add Ethernet8 10.0.4.1/24')
out = nodes['S1'].execute('sudo config interface ip add Loopback0 10.10.11.100/32')
out = nodes['S1'].execute('sudo config save -y')
print ("\n--- Verify IP address on S1 ---")
out = nodes['S1'].execute('show ip interfaces') 

In [ ]:
print ("\n--- Configuring L0 ---")
out = nodes['L0'].execute('sudo config interface ip add Ethernet0 10.0.1.2/24')
out = nodes['L0'].execute('sudo config interface ip add Ethernet12 10.0.3.2/24')
out = nodes['L0'].execute('sudo config interface ip add Ethernet8 10.0.5.1/24')
out = nodes['L0'].execute('sudo config interface ip add Loopback0 10.10.10.200/32')
out = nodes['L0'].execute('sudo config save -y')
print ("\n--- Verify IP address on L0 ---")
out = nodes['L0'].execute('show ip interfaces') 

In [ ]:
print ("\n--- Configuring L1 ---")
out = nodes['L1'].execute('sudo config interface ip add Ethernet12 10.0.2.2/24')
out = nodes['L1'].execute('sudo config interface ip add Ethernet0 10.0.4.2/24')
out = nodes['L1'].execute('sudo config interface ip add Ethernet8 10.0.6.1/24')
out = nodes['L1'].execute('sudo config interface ip add Loopback0 10.10.11.200/32')
out = nodes['L1'].execute('sudo config save -y')
print ("\n--- Verify IP address on L1 ---")
out = nodes['L1'].execute('show ip interfaces') 

The following commands configures IPv4 addresses on trex interfaces and displays the configured values:

    ifconfig <interface> <ipv4-address> netmask <subnet-mask> up
    ifconfig -a <interface>

In [ ]:
print ("\n--- Configuring TREX ---")
out = nodes['trex'].execute('ifconfig eth1 10.0.5.2 netmask 255.255.255.0 up')
out = nodes['trex'].execute('ifconfig eth2 10.0.6.2 netmask 255.255.255.0 up')
print ("\n--- Verify IP address on TREX ---")
out = nodes['trex'].execute('ifconfig -a eth1')
out = nodes['trex'].execute('ifconfig -a eth2')

### Configure eBGP

The following section shows the eBGP (exterior Border Gateway Protocol) configuration on all spine and leaf routers. The spines are in Autonomous System (AS) 100 and the leaves in AS 200. 

The following commands configures eBGP on the router:
```
vtysh
configure terminal
router-id <router-id> 
router bgp <local AS>
no bgp ebgp-requires-policy
neighbor <neighbor-ip-address> remote-as <remote-AS>
address-family ipv4 unicast
neighbor <neighbor-ip-address> allowas-in
network <network-address/subnet-mask>
redistribute connected
```

Note the usage of the ```allowas-in``` option in the ```neighbor``` command. The default behaviour of BGP is to reject learning routes from own AS. The ```allowas-in``` option overrides this default behavior and the nodes learn routes from own AS. 

In [ ]:
print ("\n--- Configuring eBGP on S0 ---")
out = nodes['S0'].execute ('''vtysh \
-c 'configure terminal' \
-c 'router-id 10.10.10.100' \
-c 'router bgp 100' \
-c 'no bgp ebgp-requires-policy' \
-c 'neighbor 10.0.2.2 remote-as 200' \
-c 'neighbor 10.0.1.2 remote-as 200' \
-c 'address-family ipv4 unicast' \
-c 'neighbor 10.0.2.2 allowas-in' \
-c 'neighbor 10.0.1.2 allowas-in' \
-c 'network 10.0.1.0/24' \
-c 'network 10.0.2.0/24' \
-c 'network 10.10.10.100/32' \
-c 'redistribute connected'
''')

In [ ]:
print ("\n--- Configuring eBGP on S1 ---")
out = nodes['S1'].execute ('''vtysh \
-c 'configure terminal' \
-c 'router-id 10.10.11.100' \
-c 'router bgp 100' \
-c 'no bgp ebgp-requires-policy' \
-c 'neighbor 10.0.3.2 remote-as 200' \
-c 'neighbor 10.0.4.2 remote-as 200' \
-c 'address-family ipv4 unicast' \
-c 'neighbor 10.0.3.2 allowas-in' \
-c 'neighbor 10.0.4.2 allowas-in' \
-c 'network 10.0.3.0/24' \
-c 'network 10.0.4.0/24' \
-c 'network 10.10.11.100/32' \
-c 'redistribute connected'
''')

In [ ]:
print ("\n--- Configuring eBGP on L0 ---")
out = nodes['L0'].execute ('''vtysh \
-c 'configure terminal' \
-c 'router-id 10.10.10.200' \
-c 'router bgp 200' \
-c 'no bgp ebgp-requires-policy' \
-c 'neighbor 10.0.1.1 remote-as 100' \
-c 'neighbor 10.0.3.1 remote-as 100' \
-c 'address-family ipv4 unicast' \
-c 'neighbor 10.0.1.1 allowas-in' \
-c 'neighbor 10.0.3.1 allowas-in' \
-c 'network 10.0.1.0/24' \
-c 'network 10.0.3.0/24' \
-c 'network 10.0.5.0/24' \
-c 'network 10.10.10.200/32' \
-c 'redistribute connected'
''')

In [ ]:
print ("\n--- Configuring eBGP on L1 ---")
out = nodes['L1'].execute ('''vtysh \
-c 'configure terminal' \
-c 'router-id 10.10.11.200' \
-c 'router bgp 200' \
-c 'no bgp ebgp-requires-policy' \
-c 'neighbor 10.0.2.1 remote-as 100' \
-c 'neighbor 10.0.4.1 remote-as 100' \
-c 'address-family ipv4 unicast' \
-c 'neighbor 10.0.2.1 allowas-in' \
-c 'neighbor 10.0.4.1 allowas-in' \
-c 'network 10.0.2.0/24' \
-c 'network 10.0.4.0/24' \
-c 'network 10.0.6.0/24' \
-c 'network 10.10.11.200/32' \
-c 'redistribute connected'
''')

Execute these commands on all routers to verify eBGP is up and routes are learnt:
```
show ip bgp
show ip route
```

In [ ]:
out = nodes['S0'].execute('''vtysh \
                          -c 'show ip bgp' \
                          ''')
out = nodes['S0'].execute('''vtysh \
                          -c 'show ip route' \
                          ''')

In [ ]:
out = nodes['S1'].execute('''vtysh \
                          -c 'show ip bgp' \
                          ''')
out = nodes['S1'].execute('''vtysh \
                          -c 'show ip route' \
                          ''')

In [ ]:
out = nodes['L0'].execute('''vtysh \
                          -c 'show ip bgp' \
                          ''')
out = nodes['L0'].execute('''vtysh \
                          -c 'show ip route' \
                          ''')

In [ ]:
out = nodes['L1'].execute('''vtysh \
                          -c 'show ip bgp' \
                          ''')
out = nodes['L1'].execute('''vtysh \
                          -c 'show ip route' \
                          ''')

Ping IP addresses on the LEAF0 (L0) router from the LEAF1 (L1) router and vice-versa using the command:
```
ping -c <count> <remote-ip-address>
```


In [ ]:

print ("\n--- Pinging from L1 to L0 ---")
out = nodes['L1'].execute('ping -c5 10.0.5.1')
print ("\n--- Pinging from L0 to L1 ---")
out = nodes['L0'].execute('ping -c5 10.0.6.1')


### Send Traffic from TREX

The code in this section generates a bidirectional traffic burst for 1 second by invoking the [TREX](https://trex-tgn.cisco.com/trex/doc/trex_manual.html#_introduction) software traffic generator. To simulate server-to-server traffic flow of a data center, the TREX software traffic generator injects traffic streams to LEAF0 and LEAF1. 

> Details of traffic stream injected into LEAF0 from TREX:
> * Source IP address: 10.0.5.2
> * Destination IP address: 10.0.6.2

> Details of traffic stream injected into LEAF1 from TREX:
> * Source IP address: 10.0.6.2
> * Destination IP address: 10.0.5.2

<br/><br/>

<center><img src="./images/3clos-traffic.png" width="700"/></center>

<br/><br/>

After code execution, check ```Total-tx-pkt``` and ```Total-rx-pkt``` in the ```summary stats``` at the end of the output to ensure that there is no traffic loss.

In [ ]:
trex_ipaddress = str(nodes['trex'].connections.cli.ip)
trex_port = str(nodes['trex'].connections.cli.port)

generate_bidir_traffic(trex_ipaddress, trex_port)

### Verify Traffic Statistics

Check the interface counters on the nodes to ensure that there is no traffic drop using the command:
```
show interface counters rif
```

In [ ]:
for n in nodes:
   if (n != 'trex'):
      out = nodes[n].execute('show interface counters rif')

You have now successfully brought up a simple 3-stage Clos network. This forms the IP-BGP underlay for your data center network. 

### Clean up Router Configurations

Clean up configurations on the router so that you can play other notebooks.
```
# Deletes IP address configured on the router interface
sudo config interface ip remove <interface> <ip-address>

# Cleans the configuration database on the router
sudo rm /etc/sonic/config_db.json

# Release the trex ports
cd /opt/cisco/trex/latest/
sudo ./dpdk_nic_bind.py --force -u <port>
sudo ./dpdk_nic_bind.py --bind=virtio-pci <port>
```

In [ ]:
# Remove the Router configs 
out = nodes['S0'].execute('sudo config interface ip remove Ethernet0 10.0.1.1/24')
out = nodes['S0'].execute('sudo config interface ip remove Ethernet8 10.0.2.1/24')
out = nodes['S0'].execute('sudo config interface ip remove Loopback0 10.10.10.100/32')
out = nodes['S1'].execute('sudo config interface ip remove Ethernet0 10.0.3.1/24')
out = nodes['S1'].execute('sudo config interface ip remove Ethernet8 10.0.4.1/24')
out = nodes['S1'].execute('sudo config interface ip remove Loopback0 10.10.11.100/32')
out = nodes['L0'].execute('sudo config interface ip remove Ethernet0 10.0.1.2/24')
out = nodes['L0'].execute('sudo config interface ip remove Ethernet12 10.0.3.2/24')
out = nodes['L0'].execute('sudo config interface ip remove Ethernet8 10.0.5.1/24')
out = nodes['L0'].execute('sudo config interface ip remove Loopback0 10.10.10.200/32')
out = nodes['L1'].execute('sudo config interface ip remove Ethernet12 10.0.2.2/24')
out = nodes['L1'].execute('sudo config interface ip remove Ethernet0 10.0.4.2/24')
out = nodes['L1'].execute('sudo config interface ip remove Ethernet8 10.0.6.1/24')
out = nodes['L1'].execute('sudo config interface ip remove Loopback0 10.10.11.200/32')


for n in nodes:
   if (n != 'trex'):
      print("cfg cleaned")
      out = nodes[n].execute('sudo rm /etc/sonic/config_db.json')
   else:
      # Release the trex ports
      out = nodes[n].execute('cd /opt/cisco/trex/latest/')
      out = nodes[n].execute('sudo ./dpdk_nic_bind.py --force -u 00:04.0')
      out = nodes[n].execute('sudo ./dpdk_nic_bind.py --force -u 00:05.0')
      out = nodes[n].execute('sudo ./dpdk_nic_bind.py --bind=virtio-pci 00:04.0')
      out = nodes[n].execute('sudo ./dpdk_nic_bind.py --bind=virtio-pci 00:05.0')

Disconnect from Sandbox VPN if you are not continuing with the other notebooks in this lab.

In [ ]:
!sudo kill -9 `pgrep openconnect`

> Let us know your feedback or queries about this notebook at mig-notebooks@cisco.com.